# Setup

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import yaml
import os

In [7]:
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

In [8]:
openai = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# Question Classifier

In [9]:
# 2. Prompt de classificação
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "Você é um classificador de intenções. "
        "Classifique a pergunta do usuário em UMA das categorias abaixo:\n"
        "- suporte\n"
        "- vendas\n"
        "- financeiro\n"
        "- outros\n\n"
        "Responda apenas com o nome da categoria."
    ),
    ("human", "{pergunta}")
])

In [10]:
chain = prompt | openai | StrOutputParser()

In [11]:
classification = chain.invoke({"pergunta": "Como faço para redefinir minha senha?"})
print(classification)

suporte


In [13]:
# elos específicos
financial_chain = PromptTemplate.from_template(
    """
    Você é um especialista financeiro.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Financeiro".
    Responda à pergunta do usuário:
    Pergunta: {query}
    Resposta:
    """
) | openai
tech_support_chain = PromptTemplate.from_template(
    """
    Você é um especialista em suporte técnico.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Técnico".
    Ajude o usuário com seu problema técnico.
    Pergunta: {query}
    Resposta:
    """
) | openai
update_registration_chain = PromptTemplate.from_template(
    """
    Você é um representante de atendimento ao cliente.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte de Cadastro".
    Guie o usuário na atualização de suas informações de cadastro.
    Pergunta: {query}
    Resposta:
    """
) | openai
other_info_chain = PromptTemplate.from_template(
    """
    Você é um assistente de informações gerais.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Geral".
    Forneça informações ao usuário sobre sua pergunta.
    Pergunta: {query}
    Resposta:
    """
) | openai

In [14]:
# Função de roteamento
def route(info):
    topic = info["topic"].lower()
    if "financeiro" in topic:
        return financial_chain
    elif "suporte" in topic:
        return tech_support_chain
    elif "vendas" in topic or "cadastro" in topic:
        return update_registration_chain
    else:
        return other_info_chain

In [15]:
response_chain = route({"topic": classification})
print(response_chain)

first=PromptTemplate(input_variables=['query'], template='\n    Você é um especialista em suporte técnico.\n    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Técnico".\n    Ajude o usuário com seu problema técnico.\n    Pergunta: {query}\n    Resposta:\n    ') last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x730066b51490>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x730066cfbb90>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')


In [16]:
#executa o objeto correto
response = response_chain.invoke({"query": "Como faço para redefinir minha senha?"})
print(response)

content='Bem-vindo ao Suporte Técnico! Para redefinir sua senha, siga estas etapas:\n\n1. Acesse a página de login do serviço ou aplicativo.\n2. Clique na opção "Esqueci minha senha" ou "Redefinir senha".\n3. Insira o endereço de e-mail associado à sua conta e envie a solicitação.\n4. Verifique seu e-mail para um link de redefinição de senha. Se não encontrar, verifique a pasta de spam ou lixo eletrônico.\n5. Clique no link e siga as instruções para criar uma nova senha.\n\nSe você tiver problemas durante o processo, não hesite em nos informar!' response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 62, 'total_tokens': 193, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'finish_reason': 'stop', 'logprobs': None} id='run-22f58e5f-

In [18]:
# Exemplo 2 (Assuntos Financeiros)
classification = chain.invoke({"pergunta": "Como posso pagar uma fatura atrasada?"})
response_chain = route({"topic": classification})
response = response_chain.invoke({"query": "Como posso pagar uma fatura atrasada?"})
print(response)

content='Bem-vindo ao Suporte Financeiro! Para pagar uma fatura atrasada, você pode seguir os seguintes passos:\n\n1. **Verifique o valor devido**: Consulte a fatura para saber o montante total, incluindo possíveis juros ou taxas por atraso.\n\n2. **Escolha o método de pagamento**: Você pode pagar a fatura através de diferentes métodos, como:\n   - Transferência bancária\n   - Cartão de crédito\n   - Pagamento online pelo site ou aplicativo da empresa\n   - Boleto bancário\n\n3. **Entre em contato com a empresa**: Se você tiver dúvidas sobre o pagamento ou se precisar de um prazo maior, entre em contato com o atendimento ao cliente da empresa. Eles podem oferecer opções de parcelamento ou renegociação.\n\n4. **Realize o pagamento**: Siga as instruções do método escolhido para efetuar o pagamento.\n\n5. **Confirme o pagamento**: Após realizar o pagamento, guarde o comprovante e verifique se a empresa atualizou seu status de pagamento.\n\nSe precisar de mais assistência, não hesite em pe

In [ ]:
# Exemplo 3 (Atualização de Cadastro)
classification = chain.invoke({"pergunta": "Preciso alterar meu endereço de e-mail."})
response_chain = route({"topic": classification})
response = response_chain.invoke({"query": "Preciso alterar meu endereço de e-mail."})
print(response)

In [ ]:
# Exemplo 4 (Outras Informações)
classification = chain.invoke({"pergunta": "Qual é a missão da empresa?"})
response_chain = route({"topic": classification})
response = response_chain.invoke({"query": "Qual é a missão da empresa?"})
print(response)